In [1]:
import pandas as pd
import time
import os
import func as func
from datetime import datetime
from binance.client import Client
from binance.enums import *

Create a client

In [2]:
def create_client(API_KEY, API_SECRET, test):
    if test:
        client = Client(API_KEY, API_SECRET, testnet=test)
        #client.API_URL = 'https://testnet.binance.vision/api'
    else:
        client = Client(API_KEY, API_SECRET)
    return client

Place order

In [3]:
def place_order(client, symbol, side, amount):
    try:
        order = client.create_order(
            symbol=symbol,
            side=side.upper(),
            type=ORDER_TYPE_MARKET,  # market Order, executed immediately at the current market price
            quantity=round(amount, 5)  # keep 5 decimal places to avoid precision errors
        )
        print(f"[{datetime.now()}] Placed {side} order: {amount} {symbol}")  # print the order
        return order
    
    except Exception as e:
        print(f"[ERROR] Order failed: {e}")
        return None

Main strategy

In [4]:
def strategy(client, df, param):
    # get the last klines to generate trading signal
    row = df.iloc[-1, :]
    # 获取账户余额
    usdt = float(client.get_asset_balance(asset="USDT")["free"])   # USDT hold
    position = float(client.get_asset_balance(asset="BTC")["free"])  # BTC hold
    pv = usdt + position * row["close"]
    order = None

    # ======== ranging market: RSI + BB ========
    if row["ADX"] < param["adx_threshold"][0]:
        if ((row["RSI"] < param["rsi_threshold"][0]) and  # RSI denotes oversold
            (row["close"] < param["bb_threshold"][0] * row["BB_low"]) and  # close falls below lower line of bbands
            (usdt / pv > 0.4)):
            allocation = min(pv * param["trade_ratio"], usdt)
            amount = allocation / row["close"]
            
            order = place_order(client, symbol, "buy", amount)

        if ((row["RSI"] > param["rsi_threshold"][1]) and
            (row["close"] > param["bb_threshold"][1] * row["BB_up"]) and
            (position > param["trade_ratio"])):
            allocation = position * row["close"]
            amount = allocation / row["close"]
            amount = min(amount, position)
                
            if amount > param["min_position"]:
                order = place_order(client, symbol, "sell", amount)

    # ======== trending market: RSI + MACD ========
    if row["ADX"] > param["adx_threshold"][1]:
        if row["MACD"] > row["signal"]:  # uptrend
            if row["RSI"] < param["rsi_up"][0]:
                allocation = min(pv * param["trade_ratio"], usdt)
                buy_price = row["close"]
                amount = allocation / buy_price
                order = place_order(client, symbol, "buy", amount)
                
            elif row["RSI"] > param["rsi_up"][1]:
                allocation = position * row["close"]
                amount = allocation / row["close"]
                amount = min(amount, position)
                    
                if amount > param["min_position"]:
                    order = place_order(client, symbol, "sell", amount)

        elif row["MACD"] < row["signal"]:  # downtrend
            if row["RSI"] < param["rsi_down"][0]:
                allocation = min(pv * param["trade_ratio"], usdt)
                amount = allocation / row["close"]

                order = place_order(client, symbol, "buy", amount)
                
            elif row["RSI"] > param["rsi_down"][1]:
                allocation = position * row["close"]
                amount = allocation / row["close"]
                amount = min(amount, position)
                    
                if amount > param["min_position"]:
                    order = place_order(client, symbol, "sell", amount)

    return order

Loop

In [5]:
def main(REAL_API_KEY, REAL_API_SECRET, TEST_API_KEY, TEST_API_SECRET, symbol, interval, param, trade_log):
    data_client = create_client(REAL_API_KEY, REAL_API_SECRET, test=False)  # getting data from live market
    trade_client = create_client(TEST_API_KEY, TEST_API_SECRET, test=True)  # trading on testnet
    
    while True:
        try:
            # get latest 200 klines
            print(f"[{datetime.now()}] Fetching data from live market")
            klines = data_client.get_klines(symbol=symbol, interval=interval, limit=200)
            df = pd.DataFrame(klines, columns=[
                "open_time", "open", "high", "low", "close", "volume", "close_time",
                "quote_asset_volume", "number_of_trades", "taker_buy_base", "taker_buy_quote", "ignore"
            ])
            df = df[["open_time", "open", "high", "low", "close", "volume", "close_time"]]  # only keep key fields
            df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
            df["close_time"] = pd.to_datetime(df["close_time"], unit="ms")
            df[['close', 'high', 'low']] = df[['close', 'high', 'low']].astype(float)  # transfer the data of selected columns from str to float
    
            # compute key indicators
            period = 14  # time interval for computing RSI and Bollinger Bands
            macd_span = [12, 26, 9]  # time interval for computing MACD
    
            df = func.compute_adx(df, period)  # compute ADX
            df['RSI'] = func.compute_rsi(df['close'], period=period, ema=True)  # compute RSI
            df['BB_sma'], df['BB_up'], df['BB_low'] = func.compute_bbands(
                df['close'], window=period)  # compute bollinger bands, keep window same as period of RSI
            df['MACD'], df['signal'], df['histogram'] = func.compute_macd(df['close'], macd_span)  # compute MACD
    
            # exercise strategy
            print(f"[{datetime.now()}] Running strategy...")
            valid_index = max(df['ADX'].first_valid_index(), df['RSI'].first_valid_index())  # find the first index of line where all indicators are not nan
            df = df[df.index >= valid_index].reset_index(drop=True)  # start from the first line that all indicators are not nan
            order = strategy(trade_client, df, param)
            if order is not None:
                record = {
                    "time": df.iloc[-1, :]["close_time"],
                    "side": order["side"],
                    "price": order["fills"][0]["price"] if "fills" in order else None,
                    "executedQty": order["executedQty"],
                    "orderId": order["orderId"]
                }
    
                # save to trades.csv
                if not os.path.exists(trade_log):
                    pd.DataFrame([record]).to_csv(trade_log, index=False)
                else:
                    pd.DataFrame([record]).to_csv(trade_log, mode="a", header=False, index=False)
                    
                print(f"[{datetime.now()}] Trade executed: {record}")
                    
        except Exception as e:
            print(f"[ERROR] {e}")
            
        time.sleep(3600)  # exercise every hour


In [5]:
def main(REAL_API_KEY, REAL_API_SECRET, TEST_API_KEY, TEST_API_SECRET, symbol, interval, param, trade_log):
    data_client = create_client(REAL_API_KEY, REAL_API_SECRET, test=False)  # getting data from live market
    trade_client = create_client(TEST_API_KEY, TEST_API_SECRET, test=True)  # trading on testnet
    
    try:
        # get latest 200 klines
        print(f"[{datetime.now()}] Fetching data from live market")
        klines = data_client.get_klines(symbol=symbol, interval=interval, limit=200)
        df = pd.DataFrame(klines, columns=[
            "open_time", "open", "high", "low", "close", "volume", "close_time",
            "quote_asset_volume", "number_of_trades", "taker_buy_base", "taker_buy_quote", "ignore"
        ])
        df = df[["open_time", "open", "high", "low", "close", "volume", "close_time"]]  # only keep key fields
        df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
        df["close_time"] = pd.to_datetime(df["close_time"], unit="ms")
        df[['close', 'high', 'low']] = df[['close', 'high', 'low']].astype(float)  # transfer the data of selected columns from str to float
    
        # compute key indicators
        period = 14  # time interval for computing RSI and Bollinger Bands
        macd_span = [12, 26, 9]  # time interval for computing MACD
    
        df = func.compute_adx(df, period)  # compute ADX
        df['RSI'] = func.compute_rsi(df['close'], period=period, ema=True)  # compute RSI
        df['BB_sma'], df['BB_up'], df['BB_low'] = func.compute_bbands(
            df['close'], window=period)  # compute bollinger bands, keep window same as period of RSI
        df['MACD'], df['signal'], df['histogram'] = func.compute_macd(df['close'], macd_span)  # compute MACD
    
        # exercise strategy
        print(f"[{datetime.now()}] Running strategy...")
        valid_index = max(df['ADX'].first_valid_index(), df['RSI'].first_valid_index())  # find the first index of line where all indicators are not nan
        df = df[df.index >= valid_index].reset_index(drop=True)  # start from the first line that all indicators are not nan
        order = strategy(trade_client, df, param)
        if order is not None:
            record = {
                "time": df.iloc[-1, :]["close_time"],
                "side": order["side"],
                "price": order["fills"][0]["price"] if "fills" in order else None,
                "executedQty": order["executedQty"],
                "orderId": order["orderId"]
            }
    
            # 保存到 trades.csv
            if not os.path.exists(trade_log):
                pd.DataFrame([record]).to_csv(trade_log, index=False)
            else:
                pd.DataFrame([record]).to_csv(trade_log, mode="a", header=False, index=False)
                    
            print(f"[{datetime.now()}] Trade executed: {record}")
                    
    except Exception as e:
        print(f"[ERROR] {e}")
            
    return klines, df

Live trading

In [6]:
if __name__ == "__main__":
    # API Key
    REAL_API_KEY = "hsG5CK4sxPq1oGKKe0CJh7NRSZok1D0BliqIxAAZhibzJ9RgeZrVIv9HhdLygvEG"  # API for getting live market data
    REAL_API_SECRET = "tx66aq8NoXFViVA7KDMUjpRWfwSl03hHTAsoTneffcEPGGC4qfC2bolVPxiHvGgQ"
    TEST_API_KEY = "RPeEh2TpFcFmasUYcQZ7DSkoIECgEnqSf0LX7uQqvw8v2D8HDWHpj6ED5mlIIk8f"  # API for testnet
    TEST_API_SECRET = "SsvFbDpyxUiyCDgNLUcUeq2feDSt3PzvBOhTvUMoEAjX1aqtjMMgkz2UxpLuPpFl"
    symbol = "BTCUSDT"  # targeted crypto
    timeframe = "1h"   # time interval
    
    # strategy parameters
    param = {
        "trade_ratio": 0.2,
        "min_position": 10e-5,  # minimum trading amount
        "adx_threshold": [20, 25],  # ranging/trending market threshold
        "rsi_threshold": [30, 70],  # oversold/overbought threshold in ranging market
        "rsi_up": [35, 75],  # oversold/overbought threshold in uptrend market
        "rsi_down": [25, 65],  # oversold/overbought threshold in downtrend market
        "bb_threshold": [0.98, 1.02]  # adjustment of bollinger band
    }
    
    # 交易日志
    trade_log="trades.csv"
    
    klines, df = main(REAL_API_KEY, REAL_API_SECRET, TEST_API_KEY, TEST_API_SECRET, symbol=symbol, interval=timeframe, param=param, trade_log=trade_log)

[2025-10-03 12:19:38.338995] Fetching data from live market
[2025-10-03 12:19:39.029723] Running strategy...


Check the state of account

In [ ]:
client = Client(TEST_API_KEY, TEST_API_SECRET, testnet=True)

account = client.get_account()
for asset in account["balances"]:
    if float(asset["free"]) > 0:
        print(asset)